In [23]:
import pandas as pd
import spacy
import re
from spacy import displacy 
from spacy.matcher import Matcher 
import visualise_spacy_tree
from IPython.display import Image, display

In [9]:
df = pd.read_csv("nbc_df.csv")
df = df.iloc[: , 1:] #get rid of redundant ids
df

,Title,Link,Date,Content
0,'We're one cigarette away': Illegal weed farms...,https://www.nbcnews.com/news/us-news/fire-guns...,Thu Dec 02 2021 01:40:21 GMT+0000 (UTC),"LAKE ELSINORE, Calif. — After a 21⁄2-mile trek..."
1,On the front lines with hotshot teams fighting...,https://www.nbcnews.com/nightly-news/video/on-...,NaN,NaN
2,Wildfire near Rocky Mountain National Park ful...,https://www.nbcnews.com/news/us-news/wildfire-...,Sun Nov 21 2021 03:09:39 GMT+0000 (UTC),A wildfire burning for four days in steep terr...
3,Thousands of giant sequoias killed in Californ...,https://www.nbcnews.com/science/science-news/t...,Fri Nov 19 2021 20:00:52 GMT+0000 (UTC),LOS ANGELES — Lightning-sparked wildfires kill...
4,Former California professor indicted on arson ...,https://www.nbcnews.com/news/us-news/former-ca...,Fri Nov 19 2021 11:57:32 GMT+0000 (UTC),A former California professor accused of setti...
...,...,...,...,...
143,"Over 900,000 acres scorched by fires in Oregon...",https://www.msnbc.com/mtp-daily/watch/over-900...,NaN,NaN
144,Power outages hamper evacuation warnings and d...,https://www.nbcnews.com/business/business-news...,Fri Sep 11 2020 17:37:16 GMT+0000 (UTC),Danielle Mays lives in a wood cabin at the end...
145,7 deaths confirmed as wildfire rages in Northe...,https://www.nbcnews.com/news/us-news/7-bodies-...,Fri Sep 11 2020 05:30:00 GMT+0000 (UTC),Seven more deaths have been confirmed where a ...
146,Wildfire north of Sacramento is now largest in...,https://www.nbcnews.com/news/us-news/wildfire-...,Fri Sep 11 2020 01:23:00 GMT+0000 (UTC),California has never before seen a fire as lar...


In [10]:
df = df.dropna().reset_index(drop=True)
df

,Title,Link,Date,Content
0,'We're one cigarette away': Illegal weed farms...,https://www.nbcnews.com/news/us-news/fire-guns...,Thu Dec 02 2021 01:40:21 GMT+0000 (UTC),"LAKE ELSINORE, Calif. — After a 21⁄2-mile trek..."
1,Wildfire near Rocky Mountain National Park ful...,https://www.nbcnews.com/news/us-news/wildfire-...,Sun Nov 21 2021 03:09:39 GMT+0000 (UTC),A wildfire burning for four days in steep terr...
2,Thousands of giant sequoias killed in Californ...,https://www.nbcnews.com/science/science-news/t...,Fri Nov 19 2021 20:00:52 GMT+0000 (UTC),LOS ANGELES — Lightning-sparked wildfires kill...
3,Former California professor indicted on arson ...,https://www.nbcnews.com/news/us-news/former-ca...,Fri Nov 19 2021 11:57:32 GMT+0000 (UTC),A former California professor accused of setti...
4,Colorado pilot fighting wildfire was making fi...,https://www.nbcnews.com/news/us-news/colorado-...,Thu Nov 18 2021 03:52:01 GMT+0000 (UTC),The pilot of a firefighting aircraft helping t...
...,...,...,...,...
108,Oregon prepares for 'mass fatality incident' a...,https://www.nbcnews.com/news/us-news/largest-w...,Sat Sep 12 2020 15:24:00 GMT+0000 (UTC),"Officials in Oregon are preparing for a ""mass ..."
109,New California law to make it easier for forme...,https://www.nbcnews.com/news/us-news/proposed-...,Fri Sep 11 2020 20:35:00 GMT+0000 (UTC),"OAKLAND, Calif.—Gov. Gavin Newsom signed a new..."
110,Power outages hamper evacuation warnings and d...,https://www.nbcnews.com/business/business-news...,Fri Sep 11 2020 17:37:16 GMT+0000 (UTC),Danielle Mays lives in a wood cabin at the end...
111,7 deaths confirmed as wildfire rages in Northe...,https://www.nbcnews.com/news/us-news/7-bodies-...,Fri Sep 11 2020 05:30:00 GMT+0000 (UTC),Seven more deaths have been confirmed where a ...


In [17]:
def clean(text):
    
    # removing paragraph numbers
    text = re.sub('[0-9]+.\t','',str(text))
    # removing new line characters
    text = re.sub('\n ','',str(text))
    text = re.sub('\n',' ',str(text))
    # removing apostrophes
    text = re.sub("'s",'',str(text))
    # removing hyphens
    text = re.sub("-",' ',str(text))
    text = re.sub("— ",'',str(text))
    # removing quotation marks
    text = re.sub('\"','',str(text))
    # removing salutations
    text = re.sub("Mr\.",'Mr',str(text))
    text = re.sub("Mrs\.",'Mrs',str(text))
    # removing any reference to outside text
    text = re.sub("[\(\[].*?[\)\]]", "", str(text))
    
    return text
df["Content"] = df["Content"].apply(func=clean)
df.rename(columns={"Content":"Clean_content"}, inplace=True)
df

,Title,Link,Date,Clean_content
0,'We're one cigarette away': Illegal weed farms...,https://www.nbcnews.com/news/us-news/fire-guns...,Thu Dec 02 2021 01:40:21 GMT+0000 (UTC),"LAKE ELSINORE, Calif. After a 21⁄2 mile trek t..."
1,Wildfire near Rocky Mountain National Park ful...,https://www.nbcnews.com/news/us-news/wildfire-...,Sun Nov 21 2021 03:09:39 GMT+0000 (UTC),A wildfire burning for four days in steep terr...
2,Thousands of giant sequoias killed in Californ...,https://www.nbcnews.com/science/science-news/t...,Fri Nov 19 2021 20:00:52 GMT+0000 (UTC),LOS ANGELES Lightning sparked wildfires killed...
3,Former California professor indicted on arson ...,https://www.nbcnews.com/news/us-news/former-ca...,Fri Nov 19 2021 11:57:32 GMT+0000 (UTC),A former California professor accused of setti...
4,Colorado pilot fighting wildfire was making fi...,https://www.nbcnews.com/news/us-news/colorado-...,Thu Nov 18 2021 03:52:01 GMT+0000 (UTC),The pilot of a firefighting aircraft helping t...
...,...,...,...,...
108,Oregon prepares for 'mass fatality incident' a...,https://www.nbcnews.com/news/us-news/largest-w...,Sat Sep 12 2020 15:24:00 GMT+0000 (UTC),Officials in Oregon are preparing for a mass f...
109,New California law to make it easier for forme...,https://www.nbcnews.com/news/us-news/proposed-...,Fri Sep 11 2020 20:35:00 GMT+0000 (UTC),"OAKLAND, Calif.—Gov. Gavin Newsom signed a new..."
110,Power outages hamper evacuation warnings and d...,https://www.nbcnews.com/business/business-news...,Fri Sep 11 2020 17:37:16 GMT+0000 (UTC),Danielle Mays lives in a wood cabin at the end...
111,7 deaths confirmed as wildfire rages in Northe...,https://www.nbcnews.com/news/us-news/7-bodies-...,Fri Sep 11 2020 05:30:00 GMT+0000 (UTC),Seven more deaths have been confirmed where a ...


In [20]:
def sentences(text):
    # split sentences and questions
    text = re.split('[.?]', text)
    clean_sent = []
    for sent in text:
        clean_sent.append(sent)
    return clean_sent

print(sentences(df["Clean_content"][1]))

['A wildfire burning for four days in steep terrain near Rocky Mountain National Park in Colorado has been fully contained after scorching 147 acres, officials said Saturday', ' The Kruger Rock Fire broke out Tuesday morning after high winds knocked down a tree onto a power line, the Larimer County Sheriff Office said', ' It became fully contained Saturday afternoon, according to the U', 'S', ' Forest Service', ' The fire near Estes Park, a town of around 6,000 residents approximately 50 miles northwest of Denver, spread quickly amid warmer than normal temperatures and drought conditions in much of Colorado and the West', 'The rugged terrain prevented firefighters from approaching by foot, prompting local officials to order a water drop by air', ' An air tanker pilot died in a crash while fighting the fire', ' The pilot, Marc Thor Olson, was attempting to drop fire suppressant over the Kruger Rock Fire when he alerted officials that conditions had become too turbulent to make the drop,

In [21]:
df["Sentences"] = df["Clean_content"].apply(sentences)
df

,Title,Link,Date,Clean_content,Sentences
0,'We're one cigarette away': Illegal weed farms...,https://www.nbcnews.com/news/us-news/fire-guns...,Thu Dec 02 2021 01:40:21 GMT+0000 (UTC),"LAKE ELSINORE, Calif. After a 21⁄2 mile trek t...","[LAKE ELSINORE, Calif, After a 21⁄2 mile trek..."
1,Wildfire near Rocky Mountain National Park ful...,https://www.nbcnews.com/news/us-news/wildfire-...,Sun Nov 21 2021 03:09:39 GMT+0000 (UTC),A wildfire burning for four days in steep terr...,[A wildfire burning for four days in steep ter...
2,Thousands of giant sequoias killed in Californ...,https://www.nbcnews.com/science/science-news/t...,Fri Nov 19 2021 20:00:52 GMT+0000 (UTC),LOS ANGELES Lightning sparked wildfires killed...,[LOS ANGELES Lightning sparked wildfires kille...
3,Former California professor indicted on arson ...,https://www.nbcnews.com/news/us-news/former-ca...,Fri Nov 19 2021 11:57:32 GMT+0000 (UTC),A former California professor accused of setti...,[A former California professor accused of sett...
4,Colorado pilot fighting wildfire was making fi...,https://www.nbcnews.com/news/us-news/colorado-...,Thu Nov 18 2021 03:52:01 GMT+0000 (UTC),The pilot of a firefighting aircraft helping t...,[The pilot of a firefighting aircraft helping ...
...,...,...,...,...,...
108,Oregon prepares for 'mass fatality incident' a...,https://www.nbcnews.com/news/us-news/largest-w...,Sat Sep 12 2020 15:24:00 GMT+0000 (UTC),Officials in Oregon are preparing for a mass f...,[Officials in Oregon are preparing for a mass ...
109,New California law to make it easier for forme...,https://www.nbcnews.com/news/us-news/proposed-...,Fri Sep 11 2020 20:35:00 GMT+0000 (UTC),"OAKLAND, Calif.—Gov. Gavin Newsom signed a new...","[OAKLAND, Calif, —Gov, Gavin Newsom signed a ..."
110,Power outages hamper evacuation warnings and d...,https://www.nbcnews.com/business/business-news...,Fri Sep 11 2020 17:37:16 GMT+0000 (UTC),Danielle Mays lives in a wood cabin at the end...,[Danielle Mays lives in a wood cabin at the en...
111,7 deaths confirmed as wildfire rages in Northe...,https://www.nbcnews.com/news/us-news/7-bodies-...,Fri Sep 11 2020 05:30:00 GMT+0000 (UTC),Seven more deaths have been confirmed where a ...,[Seven more deaths have been confirmed where a...


In [ ]:
nlp = spacy.load('en_core_web_sm',disable=['ner','textcat'])